Подключаем необходимыые библиотеки

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report, mean_squared_error, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from joblib import Parallel, delayed
from collections import Counter





Загружаем датасет для задачи классификации

In [2]:
data=pd.read_csv('machine_failure_dataset.csv')

Информация о датасете

In [3]:
data.head()

,Temperature,Vibration,Power_Usage,Humidity,Machine_Type,Failure_Risk
0,74.967142,56.996777,8.649643,20.460962,Mill,1
1,68.617357,54.623168,9.710963,25.698075,Lathe,0
2,76.476885,50.298152,8.415160,27.931972,Drill,1
3,85.230299,46.765316,9.384077,39.438438,Lathe,1
4,67.658466,53.491117,6.212771,32.782766,Drill,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Temperature   1000 non-null   float64
 1   Vibration     1000 non-null   float64
 2   Power_Usage   1000 non-null   float64
 3   Humidity      1000 non-null   float64
 4   Machine_Type  1000 non-null   object 
 5   Failure_Risk  1000 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 47.0+ KB


Проверяем на наличие нулевых ячееек и дубликатов

In [5]:
data.isnull().sum()

Temperature     0
Vibration       0
Power_Usage     0
Humidity        0
Machine_Type    0
Failure_Risk    0
dtype: int64

In [6]:
data.duplicated().sum()

0

Убираем колонку Machine_Type

In [7]:
data = pd.get_dummies(data,columns = ['Machine_Type'],drop_first = True)

Делим датасет на фичи (x) и таргет (y) 

In [8]:
x = data.drop(['Failure_Risk'],axis = 1)
y = data.Failure_Risk

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

Делим датасет на тренировочный и тестовый

In [10]:
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.2,random_state=42)

Загружаем датасет для задачи регрессии и смотрим информацию о нем

In [11]:
df = pd.read_csv("Sleep_Efficiency.csv")
df.head()

,ID,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
0,1,65,Female,2021-03-06 01:00:00,2021-03-06 07:00:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0
1,2,69,Male,2021-12-05 02:00:00,2021-12-05 09:00:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
2,3,40,Female,2021-05-25 21:30:00,2021-05-25 05:30:00,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0
3,4,40,Female,2021-11-03 02:30:00,2021-11-03 08:30:00,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0
4,5,57,Male,2021-03-13 01:00:00,2021-03-13 09:00:00,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0


In [12]:
mode_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')
df['Awakenings'] = mode_imputer.fit_transform(df[['Awakenings']])
df['Exercise frequency'] = mode_imputer.fit_transform(df[['Exercise frequency']])
df['Caffeine consumption'] = mean_imputer.fit_transform(df[['Caffeine consumption']])
df['Alcohol consumption'] = mean_imputer.fit_transform(df[['Alcohol consumption']])

Убираем дупликаты

In [13]:
df.drop_duplicates(subset='ID')
df=df.drop(columns='ID', axis=1)

Преобразовываем значения в столбце Smoking status в значения 1, 0

In [14]:
df['Smoking status'] = df['Smoking status'].map({'Yes': 1, 'No': 0})

Убираем столбец Gender

In [15]:
df=pd.concat([df,pd.get_dummies(df['Gender'])], axis=1)
df=df.drop('Gender', axis=1)

Перевод столбцов Bedtime и Wakeup time в формат

In [16]:
df['Bedtime'] = pd.to_datetime(df['Bedtime']).dt.hour
df['Wakeup time'] = pd.to_datetime(df['Wakeup time']).dt.hour

In [17]:
df['Caffeine consumption'] = df['Caffeine consumption'].round(2)

Делим датасет на фичи (w) и таргет (z) и делим их на тестовый и тренировачный

In [18]:
z = df.drop('Sleep efficiency', axis=1)
w = df['Sleep efficiency']

In [19]:
scaler = MinMaxScaler()
z_scaled = scaler.fit_transform(z)

In [20]:
z_train, z_test, w_train, w_test = train_test_split(z_scaled, w, test_size=0.2,random_state=42)

#                                               LAB 1 -  KNN 

#### Создание бейзлайна и оценка качества для модели KNN

In [21]:
knn_clas = KNeighborsClassifier(n_neighbors=5)
knn_clas.fit(x_train, y_train)
y_pred = knn_clas.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.625


In [22]:
knn_reg = KNeighborsRegressor()
knn_reg.fit(z_train, w_train)
w_pred = knn_reg.predict(z_test)
print("MSE:", mean_squared_error(w_test, w_pred))

MSE: 0.0032619340659340653


#### Улучшение бейзлайна за счет подбора гиперпараметров

In [23]:
params = {'n_neighbors': [1, 2, 3, 4, 5], 'weights': ['uniform', 'distance']}
grid_class = GridSearchCV(KNeighborsClassifier(), params, cv=5, scoring='accuracy')
grid_class.fit(x_train, y_train)
knn_clas_grid = grid_class.best_estimator_
y_pred = knn_clas_grid.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.655


In [59]:
params = {'n_neighbors': [1, 5], 'weights': ['uniform', 'distance']}
grid_reg = GridSearchCV(KNeighborsRegressor(), params, cv=5, scoring='neg_mean_squared_error')
grid_reg.fit(z_train, w_train)
knn_reg_grid = grid_reg.best_estimator_
w_pred = knn_reg_grid.predict(z_test)
print("MSE:", mean_squared_error(w_test, w_pred))

MSE: 0.003253538683766209


#### Реализация алгоритма KNN

In [25]:
class KNN:
    
    def __init__(self, k=3, metric="euclidean", mode="classification"):
        self.k = k
        self.metric = metric
        self.mode = mode
        self.X_train = None
        self.y_train = None
        
    def _euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2)**2))
    
    def _manhattan_distance(self, x1, x2):
        return np.sum(np.abs(x1 - x2))
        
    def _calculate_distance(self, x1, x2):
         if self.metric == "euclidean":
             return self._euclidean_distance(x1, x2)
         elif self.metric == "manhattan":
              return self._manhattan_distance(x1,x2)
    def fit(self, X, y):
        self.X_train = np.array(X) 
        self.y_train = np.array(y)
        
    def predict_point(self, x):
        
        distances = [self._calculate_distance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k] 
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        
        if self.mode == "classification":
            most_common = Counter(k_nearest_labels).most_common(1) 
            return most_common[0][0] 
        elif self.mode == "regression":
            return np.mean(k_nearest_labels) 
        
    def predict(self, X):
       
        return np.array([self.predict_point(x) for x in X])

    def evaluate(self, X, y_true):
        y_pred = self.predict(X)

        if self.mode == "classification":
             return accuracy_score(y_true, y_pred)
        elif self.mode == "regression":
            return mean_squared_error(y_true, y_pred)
     



#### Обучение модели о оценка качества

In [26]:
model_clas = KNN(k=3, metric="euclidean", mode="classification")
model_clas.fit(x_train, y_train)
accuracy = model_clas.evaluate(x_test, y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.59


In [27]:
model_reg = KNN(k=2, metric="manhattan", mode="regression")
model_reg.fit(z_train, w_train)
mse = model_reg.evaluate(z_test, w_test)
print(f"MSE: {mse}")

MSE: 0.004254395604395606


# LAB 2 - LOGISTIC AND LINEAR REGRESSION

#### Создание бейзлайна и оценка качества для модели логистической и линейной регрессии

In [28]:
lr_clas = LogisticRegression()
lr_clas.fit(x_train, y_train)
y_pred = lr_clas.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.675


In [29]:
lr_reg = LinearRegression()
lr_reg.fit(x_train, y_train)
y_pred = lr_reg.predict(x_test)
print("MSE:", mean_squared_error(y_test, y_pred))

MSE: 0.22627973936551224


#### Улучшение бейзлайна за счет подбора гиперпараметров

In [30]:
params = {'C': [0.01, 0.1, 1, 10, 75, 1000]}
grid_clas = GridSearchCV(LogisticRegression(), params, cv=5, scoring='accuracy')
grid_clas.fit(x_train, y_train)
lr_clas_gris = grid_clas.best_estimator_
y_pred = lr_clas_gris.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.675


In [31]:
params = {'alpha': [0.1, 1, 10, 100]}
grid_reg = GridSearchCV(Ridge(), params, cv=5, scoring='neg_mean_squared_error')
grid_reg.fit(x_train, y_train)
lr_reg_gris = grid_reg.best_estimator_
y_pred = lr_reg_gris.predict(x_test)
print("MSE:", mean_squared_error(y_test, y_pred))

MSE: 0.22126252774741673


#### Реализация алгоритма линейной регрессии

In [32]:
class LinearRegression:
   
    def __init__(self, learning_rate=0.01, n_iterations=1000, fit_intercept=True):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.fit_intercept = fit_intercept
        self.weights = None
    
    def _add_intercept(self, X):
       
        if self.fit_intercept:
            intercept = np.ones((X.shape[0], 1))
            return np.concatenate((intercept, X), axis=1)
        return X
    
    def _sigmoid(self, z):
 
        return 1 / (1 + np.exp(-z))
        
    def _gradient_descent_regression(self, X, y):
       
        m = X.shape[0]
        
        for _ in range(self.n_iterations):
            y_predicted = np.dot(X, self.weights)
            dw = (1 / m) * np.dot(X.T, (y_predicted - y))
            self.weights -= self.learning_rate * dw
    
    def _gradient_descent_classification(self, X, y):
       
        m = X.shape[0]
        
        for _ in range(self.n_iterations):
            z = np.dot(X, self.weights)
            y_predicted = self._sigmoid(z)
            dw = (1 / m) * np.dot(X.T, (y_predicted - y))
            self.weights -= self.learning_rate * dw

    def fit(self, X, y, mode="regression"):
        
        X = self._add_intercept(np.array(X)) 
        y = np.array(y) 
        
        self.weights = np.zeros(X.shape[1]) 
        
        if mode == "regression":
             self._gradient_descent_regression(X, y)
        elif mode == "classification":
            self._gradient_descent_classification(X,y)
       
    def predict(self, X, mode="regression"):
        
        X = self._add_intercept(np.array(X))
        
        if mode == "regression":
             return np.dot(X, self.weights)
        elif mode == "classification":
             z = np.dot(X, self.weights)
             return np.round(self._sigmoid(z)) 
        
    def evaluate(self, X, y_true, mode="regression"):
       
        y_pred = self.predict(X, mode)
        
        if mode == "regression":
             return mean_squared_error(y_true, y_pred)
        elif mode == "classification":
             return accuracy_score(y_true, y_pred)

#### Обучение модели о оценка качества

In [33]:
model_clas = LinearRegression(learning_rate=0.01, n_iterations=1000, fit_intercept=True)
model_clas.fit(x_train, y_train, mode="classification")
accuracy = model_clas.evaluate(x_test, y_test, mode="classification")
print(f"Accuracy: {accuracy}")

Accuracy: 0.675


In [34]:
model_reg = LinearRegression(learning_rate=0.01, n_iterations=1000, fit_intercept=True)
model_reg.fit(z_train, w_train, mode="regression")
mse = model_reg.evaluate(z_test, w_test, mode="regression")
print(f"MSE: {mse}")

MSE: 0.0058883153243692


# LAB 3 - DECISION TREE

#### Создание бейзлайна и оценка качества для модели Desicion Tree

In [35]:
dt_clas = DecisionTreeClassifier()
dt_clas.fit(x_train, y_train)
y_pred = dt_clas.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.52


In [36]:
dt_reg = DecisionTreeRegressor()
dt_reg.fit(z_train, w_train)
w_pred = dt_reg.predict(z_test)
print("MSE:", mean_squared_error(w_test, w_pred))

MSE: 0.006548351648351651


#### Улучшение бейзлайна за счет подбора гиперпараметров

In [37]:
params = {'max_depth': [3, 5, 10]}
grid_clas = GridSearchCV(DecisionTreeClassifier(), params, cv=5, scoring='accuracy')
grid_clas.fit(x_train, y_train)
dt_grid_clas = grid_clas.best_estimator_
y_pred = dt_grid_clas.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.665


In [38]:
params = {'max_depth': [3, 5, 10, 15]}
grid_reg = GridSearchCV(DecisionTreeRegressor(), params, cv=5, scoring='neg_mean_squared_error')
grid_reg.fit(z_train, w_train)
dt_grid_reg = grid_reg.best_estimator_
w_pred = dt_grid_reg.predict(z_test)
print("MSE:", mean_squared_error(w_test, w_pred))

MSE: 0.0028024622225629402


#### Реализация алгоритма Decision Tree

In [39]:
class DecisionTree:
   
    def __init__(self, max_depth=None, min_samples_split=2, mode="classification"):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.mode = mode
        self.tree = {}

    def _calculate_entropy(self, y):
        
        labels, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))

    def _calculate_gini(self, y):
        
        labels, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities**2)
    
    def _calculate_variance(self, y):
        return np.var(y)

    def _information_gain(self, y, y_left, y_right):
        
       n = len(y)
       n_left, n_right = len(y_left), len(y_right)
       if n_left == 0 or n_right == 0:
           return 0 #если один из наборов пустой
       
       if self.mode == "classification":
          parent_loss = self._calculate_entropy(y) if len(np.unique(y)) > 1 else 0 # если в одном классе, то энтропия 0
          child_loss = (n_left / n) * self._calculate_entropy(y_left) + (n_right/n) * self._calculate_entropy(y_right)
       elif self.mode == "regression":
           parent_loss = self._calculate_variance(y)
           child_loss = (n_left / n) * self._calculate_variance(y_left) + (n_right/n) * self._calculate_variance(y_right)
        

       return parent_loss - child_loss


    def _best_split(self, X, y):
    
        best_gain = 0
        best_feature = None
        best_threshold = None
        
        for feature in range(X.shape[1]):
            unique_values = np.unique(X[:, feature])
            for threshold in unique_values:
                mask = X[:, feature] <= threshold
                y_left = y[mask]
                y_right = y[~mask]
                gain = self._information_gain(y, y_left, y_right)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
                    
        return best_feature, best_threshold

    def _build_tree(self, X, y, depth=0):
       
        num_samples = X.shape[0]
        num_labels = len(np.unique(y))
        
        # базовый случай 1: достигнута макс. глубина дерева, либо меньше сэмплов, чем необходимо для разделения
        if (self.max_depth is not None and depth >= self.max_depth) or \
           num_samples < self.min_samples_split or num_labels == 1:
                if self.mode == "classification":
                    return Counter(y).most_common(1)[0][0]
                elif self.mode == "regression":
                    return np.mean(y)
                
        
        best_feature, best_threshold = self._best_split(X, y)

        if best_feature is None:
            if self.mode == "classification":
                    return Counter(y).most_common(1)[0][0]
            elif self.mode == "regression":
                    return np.mean(y)
            
        
        mask = X[:, best_feature] <= best_threshold
        X_left = X[mask]
        y_left = y[mask]
        X_right = X[~mask]
        y_right = y[~mask]

        subtree = {"feature": best_feature, "threshold": best_threshold, "left": None, "right": None}
        subtree["left"] = self._build_tree(X_left, y_left, depth + 1)
        subtree["right"] = self._build_tree(X_right, y_right, depth + 1)
        return subtree

    def fit(self, X, y):
       
        self.tree = self._build_tree(np.array(X), np.array(y))

    def _predict_point(self, x, tree):
       
       if isinstance(tree, dict): 
            if x[tree['feature']] <= tree['threshold']:
               return self._predict_point(x, tree['left'])
            else:
                return self._predict_point(x, tree['right'])
       else: 
          return tree
       
    def predict(self, X):
       
        return np.array([self._predict_point(x, self.tree) for x in X])

    def evaluate(self, X, y_true):
        
       y_pred = self.predict(X)
       
       if self.mode == "classification":
            return accuracy_score(y_true, y_pred)
       elif self.mode == "regression":
           return mean_squared_error(y_true, y_pred)


#### Обучение модели о оценка качества

In [40]:
model_clas = DecisionTree(max_depth=3, mode="classification")
model_clas.fit(x_train, y_train)
accuracy = model_clas.evaluate(x_test, y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.675


In [41]:
model_reg = DecisionTree(max_depth=3, mode="regression")
model_reg.fit(z_train, w_train)
mse = model_reg.evaluate(z_test, w_test)
print(f"MSE: {mse}")

MSE: 0.0035214200324298456


# LAB 4 - RANDOM FOREST

#### Создание бейзлайна и оценка качества для модели Random Forest

In [42]:
rf_clas = RandomForestClassifier()
rf_clas.fit(x_train, y_train)
y_pred = rf_clas.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.655


In [43]:
rf_reg = RandomForestRegressor()
rf_reg.fit(z_train, w_train)
w_pred = rf_reg.predict(z_test)
print("MSE:", mean_squared_error(w_test, w_pred))

MSE: 0.0027514770329670397


#### Улучшение бейзлайна за счет подбора гиперпараметров

In [44]:
params = {'n_estimators': [50, 100], 'max_depth': [10, 15]}
grid__clas = GridSearchCV(RandomForestClassifier(), params, cv=5, scoring='accuracy')
grid_clas.fit(x_train, y_train)
rf_grid_reg = grid_clas.best_estimator_
y_pred = rf_grid_reg.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.665


In [45]:
params = {'n_estimators': [50, 100], 'max_depth': [5, 10, 15, 20]}
grid_reg = GridSearchCV(RandomForestRegressor(), params, cv=5, scoring='neg_mean_squared_error')
grid_reg.fit(z_train, w_train)
rf_grid_reg = grid_reg.best_estimator_
w_pred = rf_grid_reg.predict(z_test)
print("MSE:", mean_squared_error(w_test, w_pred))

MSE: 0.002458735994456027


#### Реализация алгоритма Random Forest

In [46]:
class RandomForest:
    def __init__(self, n_trees=10, max_depth=3, sample_size=0.8, mode="classification"):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.sample_size = sample_size
        self.mode = mode
        self.trees = []

    def fit(self, X, y):
        self.X = np.array(X)
        self.y = np.array(y)
        for _ in tqdm(range(self.n_trees), desc="Training Trees"):
            indices = np.random.choice(len(self.X), int(len(self.X) * self.sample_size), replace=True)
            X_sample, y_sample = self.X[indices], self.y[indices]
            tree = DecisionTree(max_depth=self.max_depth, mode=self.mode)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        if self.mode == "classification":
            return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)
        else:
            return np.mean(predictions, axis=0)
    def evaluate(self, X, y_true):
        
       y_pred = self.predict(X)
       
       if self.mode == "classification":
            return accuracy_score(y_true, y_pred)
       elif self.mode == "regression":
           return mean_squared_error(y_true, y_pred)
    
    

#### Обучение модели о оценка качества

In [47]:
model_clas = RandomForest(max_depth=3, mode="classification")
model_clas.fit(x_train, y_train)
accuracy = model_clas.evaluate(x_test, y_test)
print(f"Accuracy: {accuracy}")

Training Trees: 100%|███████████████████████████| 10/10 [00:08<00:00,  1.16it/s]

Accuracy: 0.675


In [48]:
model_reg = RandomForest(max_depth=3, mode="regression")
model_reg.fit(z_train, w_train)
mse = model_reg.evaluate(z_test, w_test)
print(f"MSE: {mse}")

Training Trees: 100%|███████████████████████████| 10/10 [00:00<00:00, 16.42it/s]

MSE: 0.003425663087114921


# LAB 5 - GRADIENT BOOSTING

#### Создание бейзлайна и оценка качества для модели Gradint Boosting

In [49]:
gb_clas = GradientBoostingClassifier()
gb_clas.fit(x_train, y_train)
y_pred = gb_clas.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.63


In [50]:
gb_reg = GradientBoostingRegressor()
gb_reg.fit(z_train, w_train)
w_pred = gb_reg.predict(z_test)
print("MSE:", mean_squared_error(w_test, w_pred))

MSE: 0.0026539676331362905


#### Улучшение бейзлайна за счет подбора гиперпараметров

In [51]:
params = {'n_estimators': [50, 100], 'max_depth': [3, 5], 'learning_rate': [0.1, 0.01]}
grid_clas = GridSearchCV(GradientBoostingClassifier(), params, cv=5, scoring='accuracy')
grid_clas.fit(x_train, y_train)
gb_grid_clas = grid_clas.best_estimator_
y_pred = gb_grid_clas.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.675


In [52]:
params = {'n_estimators': [50, 100], 'max_depth': [3, 5], 'learning_rate': [0.1, 0.01]}
grid_reg = GridSearchCV(GradientBoostingRegressor(), params, cv=5, scoring='neg_mean_squared_error')
grid_reg.fit(z_train, w_train)
gb_grid_reg = grid_reg.best_estimator_
w_pred = gb_grid_reg.predict(z_test)
print("MSE:", mean_squared_error(w_test, w_pred))

MSE: 0.002546143547670316


#### Реализация алгоритма Gradient Boosting

In [53]:
class GradientBoosting:
  

    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, loss="mse"):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.loss = loss
        self.init_prediction = None
        self.trees = []

    def _calculate_gradient(self, y_true, y_pred):
      
        if self.loss == "mse":
            return y_true - y_pred
        elif self.loss == "logloss":
            
            sigmoid = 1 / (1 + np.exp(-y_pred))
            return y_true - sigmoid
    
    def fit(self, X, y):
        
        
        if self.loss == "logloss":
           
            unique_labels = np.unique(y)
            if len(unique_labels) > 2:
                raise ValueError("Для 'logloss' целевая переменная должна быть бинарной")
            
            self.init_prediction = np.log(np.sum(y == unique_labels[1]) / np.sum(y == unique_labels[0])) 
        else:
            self.init_prediction = np.mean(y)
        
        
        y_pred = np.full_like(y, self.init_prediction, dtype=float) 

        for _ in range(self.n_estimators):
        
            residuals = self._calculate_gradient(y, y_pred)

           
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
        
            y_pred += self.learning_rate * tree.predict(X)

           
            self.trees.append(tree)

    def predict(self, X):
       
        y_pred = np.full(X.shape[0], self.init_prediction, dtype=float)

        for tree in self.trees:
            y_pred += self.learning_rate * tree.predict(X)
        
        if self.loss == "logloss":
            return 1 / (1 + np.exp(-y_pred)) 
        else:
            return y_pred

    def evaluate(self, X, y_true):
        
        y_pred = self.predict(X)
        
        if self.loss == "mse":
            return mean_squared_error(y_true, y_pred)
        elif self.loss == "logloss":
             y_pred_binary = np.round(y_pred)  # Преобразуем вероятности в классы 0 или 1
             return accuracy_score(y_true, y_pred_binary)
        



#### Обучение модели о оценка качества

In [54]:
model_clas = GradientBoosting(n_estimators=100, learning_rate=0.1, max_depth=3, loss="logloss")
model_clas.fit(x_train, y_train)
accuaracy = model_clas.evaluate(x_test, y_test)
print(f"Accuaracy: {accuaracy}")

Accuaracy: 0.675


In [55]:
model_reg = GradientBoosting(n_estimators=100, learning_rate=0.1, max_depth=3, loss="mse")
model_reg.fit(z_train, w_train)
mse = model_reg.evaluate(z_test, w_test)
print(f"Регрессия MSE: {mse}")

Регрессия MSE: 0.002677959447156116
